## Notebook for spike sorting from .rhd data using the Pipeline of mountainsort (linux channel has pipeline)
Uses:
    - intan2kwik (https://github.com/zekearneodo/intan2kwik/blob/master/README.md)
    - mountainlab suite(https://github.com/flatironinstitute/mountainlab-js)
    - mountainsort https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md
    - mountainsort examples https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md

In [1]:
import os
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import subprocess
from importlib import reload

# mountainlab imports
from pipefinch.neural.convert import intan
from pipefinch.neural.sort.mountain import core as msc

import logging

# Setup the logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(formatter)
logger.addHandler(ch)
        
logger.info('Logger set')

/home/zinch/conda/envs/msort/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/zinch/conda/envs/msort/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/zinch/conda/envs/msort/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)
2018-12-17 15:27:29,283 - root - INFO - Logger set


In [2]:
os.environ['OMP_NUM_THREADS'] = '8'
os.environ['MKL_NUM_THREADS'] = '8'
os.environ['NUMEXPR_NUM_THREADS'] = '8'

### Session parameters and raw files

In [3]:
exp_base = os.path.abspath('/home/zinch/data/')
exp_mnt = os.path.abspath('/mnt/zuperfinchjr/Data/')

file_sys = {'exp_base': exp_base}

sess_par = {'bird': 'o9j9',
           'sess': '20181125_r021_saveontrigger',
            'neural': 'Neural'}


def file_names(sess_par, file_sys=None):
    f_s = unroll_file_sys(file_sys_defaults, file_sys)    
    
    f_n = {}
    f_n['raw'] = os.path.join(f_s['raw'], bird)

In [4]:
exp_base

'/home/zinch/data'

In [5]:
raw_folder = os.path.join(exp_mnt, sess_par['bird'], sess_par['neural'], sess_par['sess'])
logger.info('Raw folder {}'.format(raw_folder))
#glob.glob(os.path.join(raw_folder, '*.rhd'))

2018-12-17 15:27:39,000 - root - INFO - Raw folder /mnt/zuperfinchjr/Data/o9j9/Neural/20181125_r021_saveontrigger


In [6]:
rhd_files = glob.glob(os.path.join(raw_folder, '*.rhd'))
rhd_files.sort()
pd_f = pd.DataFrame(rhd_files, columns=['f_path'])

pd_f['stamp'] = pd_f['f_path'].apply(lambda x: int(os.path.split(x)[-1].split('.')[0].split('_')[-1]))

# Select some of the files
sess_start = 0
sess_end = 235959
pd_session = pd_f.loc[(pd_f['stamp'] > sess_start) & (pd_f['stamp'] < sess_end), :]
pd_session.index.size

rhd_list = pd_session['f_path'].values.tolist()[:]

## Scripts for sorting

### prep the files with their nice formats, locations and names


In [7]:
rhd_list

['/mnt/zuperfinchjr/Data/o9j9/Neural/20181125_r021_saveontrigger/o9j9_r020_saveontrigger_181125_132219.rhd',
 '/mnt/zuperfinchjr/Data/o9j9/Neural/20181125_r021_saveontrigger/o9j9_r020_saveontrigger_181125_132319.rhd',
 '/mnt/zuperfinchjr/Data/o9j9/Neural/20181125_r021_saveontrigger/o9j9_r020_saveontrigger_181125_132419.rhd',
 '/mnt/zuperfinchjr/Data/o9j9/Neural/20181125_r021_saveontrigger/o9j9_r020_saveontrigger_181125_132519.rhd',
 '/mnt/zuperfinchjr/Data/o9j9/Neural/20181125_r021_saveontrigger/o9j9_r020_saveontrigger_181125_132619.rhd',
 '/mnt/zuperfinchjr/Data/o9j9/Neural/20181125_r021_saveontrigger/o9j9_r020_saveontrigger_181125_132719.rhd',
 '/mnt/zuperfinchjr/Data/o9j9/Neural/20181125_r021_saveontrigger/o9j9_r020_saveontrigger_181125_132819.rhd',
 '/mnt/zuperfinchjr/Data/o9j9/Neural/20181125_r021_saveontrigger/o9j9_r020_saveontrigger_181125_132919.rhd',
 '/mnt/zuperfinchjr/Data/o9j9/Neural/20181125_r021_saveontrigger/o9j9_r020_saveontrigger_181125_133019.rhd',
 '/mnt/zuperfinchjr

In [23]:
f1 = rhd_list[0]
('').join(os.path.split(f1)[-1].split('_')[-2:]).split('.')[0]

'181125132219'

In [8]:
ss_folder = os.path.join(exp_base, sess_par['bird'], 'ss_data', sess_par['sess'])

file_paths, out_folder = msc.make_paths(ss_folder)
os.makedirs(ss_folder, exist_ok=True)
#todo:
# meta and pars from intan files et al
s_f = 20000

# parameters to pass to the msort scripts
ds_params = {'samplerate': s_f,
            'detect_sign': -1}

with open(file_paths['params'], 'w') as fp:
    json.dump(ds_params, fp)
    logger.info('Created session par files {}'.format(file_paths['params']))
    
# create the mda file from the rhd_list
#intan_meta, samples_in = intan.intan_to_bin(rhd_list[:], file_paths['mda'], variant='mda');


2018-12-17 15:29:21,156 - root - INFO - Created session par files /home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/params.json


In [9]:
ds_params

{'samplerate': 20000, 'detect_sign': -1}

In [10]:
file_paths

{'mda': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/raw.mda',
 'params': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/params.json',
 'geom': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/geom.csv',
 'filt': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/filt.mda.prv',
 'pre': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/pre.mda.prv',
 'firings': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/firings.mda',
 'firings_curated': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/firings_curated.mda',
 'cluster_metrics': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/cluster_metrics.json'}

In [11]:
# test mountainsort method for reading sorting parameters
msc.read_dataset_params(ss_folder)

{'samplerate': 20000, 'detect_sign': -1}

In [12]:
# Initialize a pipeline
from mountainlab_pytools import mlproc as mlp
Pipeline=mlp.initPipeline();

JSProxyWidget(status='Not yet rendered')

In [15]:
msc.sort_dataset?

Signature: msc.sort_dataset(*, file_paths:dict, freq_min:int=300, freq_max:int=6000, adjacency_radius:int=1, detect_threshold:float=3, dispatch_method:str='run', opts:dict={})
Docstring: <no docstring>
File:      ~/repos/pipefinch/pipefinch/neural/sort/mountain/core.py
Type:      function


In [16]:
output_dir = os.path.join(ss_folder, 'sort_out');
# Pipeline was failing, need to debug this ('NameError: name 'widgets' is not defined')
# with Pipeline:
#     msc.sort_dataset(file_paths=file_paths, adjacency_radius=-1, detect_threshold=3, dispatch_method='add')

msc.sort_dataset(file_paths=file_paths, adjacency_radius=-1, detect_threshold=2, freq_min=600);

2018-12-06 15:34:24,142 - pipefinch.sort.mountain.comre - INFO - Bandpass filter


RUNNING: ml-run-process ephys.bandpass_filter --inputs timeseries:/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/raw.mda --parameters freq_max:6000 freq_min:600 samplerate:20000 --outputs timeseries_out:/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/filt.mda.prv


2018-12-06 15:38:36,371 - pipefinch.sort.mountain.comre - INFO - Whitening


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
Process signature: 8f2427a3a5e7d84a7319c5bcb67b7f5ee02a605d
[ Checking outputs... ]
{"timeseries_out":"/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/filt.mda.prv"}
Processing ouput - /home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/filt.mda.prv
false
{"timeseries_out":"/tmp/mountainlab-tmp/output_8f2427a3a5e7d84a7319c5bcb67b7f5ee02a605d_timeseries_out.mda"}
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Creating links to input files... ]
[ Preparing temporary outputs... ]
Processing ouput - /tmp/mountainlab-tmp/output_8f2427a3a5e7d84a7319c5bcb67b7f5ee02a605d_timeseries_out.mda
false
[ Initializing process ... ]
[ Running ... ] /home/zinch/conda/envs/msort/bin/python3 /home/zinch/conda/envs/msort/etc/mountainlab/packages/ml_ephys/preprocessing/preprocessing.py.mp ephys.bandpass_filter --_tempdir=/tmp/mountainlab-tmp

2018-12-06 15:40:17,958 - pipefinch.sort.mountain.comre - INFO - Sorting


...

RUNNING: ml-run-process ms4alg.sort --inputs timeseries:/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/pre.mda.prv --parameters adjacency_radius:-1 detect_sign:-1 detect_threshold:2 --outputs firings_out:/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/firings.mda
[ Getting processor spec... ]
/home/zinch/conda/envs/msort/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
[ Checking inputs and substituting prvs ... ]
return f(*args, **kwds)
[ Computing process signature ... ]



2018-12-06 16:06:08,588 - pipefinch.sort.mountain.comre - INFO - Getting cluster metrics


Process signature: 5d78541c7eb38312b7b3efa04a70139d1323169a
[ Checking outputs... ]
{"firings_out":"/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/firings.mda"}
Processing ouput - /home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/firings.mda
false
{"firings_out":"/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/firings.mda"}
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Creating links to input files... ]
[ Preparing temporary outputs... ]
Processing ouput - /home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/firings.mda
false
[ Initializing process ... ]
[ Running ... ] /home/zinch/conda/envs/msort/bin/python3 /home/zinch/conda/envs/msort/etc/mountainlab/packages/ml_ms4alg/ms4alg_spec.py.mp ms4alg.sort --_tempdir=/tmp/mountainlab-tmp/tempdir_5d78541c7e_cWoI4S --timeseries=/tmp/mountainlab-tmp/tempdir_5d78541c7e_cWoI4S/input_timeseries_2PQcRCuD.mda --geom= --firings_out=/tmp/mountainlab-tmp/tem

2018-12-06 16:08:25,247 - pipefinch.sort.mountain.comre - INFO - Automatically curating


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
Process signature: db2879f75f6042453a0d5822eafc62f0ab3d6d31
[ Checking outputs... ]
{"metrics_out":"/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/cluster_metrics.json"}
Processing ouput - /home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/cluster_metrics.json
false
{"metrics_out":"/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/cluster_metrics.json"}
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Creating links to input files... ]
[ Preparing temporary outputs... ]
Processing ouput - /home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/cluster_metrics.json
false
[ Initializing process ... ]
[ Running ... ] /home/zinch/conda/envs/msort/etc/mountainlab/packages/ms3.mp ms3.combine_cluster_metrics --_tempdir=/tmp/mountainlab-tmp/tempdir_db2879f75f_vTkHuD --metrics_list=/tmp/mountainlab-tm

In [18]:
file_paths

{'mda': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/raw.mda',
 'params': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/params.json',
 'geom': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/geom.csv',
 'filt': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/filt.mda.prv',
 'pre': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/pre.mda.prv',
 'firings': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/firings.mda',
 'firings_curated': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/firings_curated.mda',
 'cluster_metrics': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/cluster_metrics.json'}

## Command for viewing:
 - open up terminal with the environment msort
 - go go the ss_data folder for the session
 - run the command: qt-mountainview --raw raw.mda --filt sort_out/filt.mda.prv --pre sort_out/pre.mda.prv --samplerate=20000 --firings sort_out/firings.mda --cluster_metrics sort_out/cluster_metrics.json